In [ ]:
import pandas as pd
import json
import torch
from transformers import LayoutLMv2FeatureExtractor,AutoTokenizer

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/opt/ml/docvqa/data/train/train_v1.0.json", field="data")

In [1]:
# If Error in "feature_extractor". In Ternimal Type : apt install tesseract-ocr
from transformers import LayoutLMv2FeatureExtractor
from PIL import Image

feature_extractor = LayoutLMv2FeatureExtractor(apply_ocr=False)

def normalize_bbox(bboxes, width, height):
    x_min = min(bboxes[::2])
    x_max = max(bboxes[::2])
    y_min = min(bboxes[1::2])
    y_max = max(bboxes[1::2])
    a = [bboxes[0],bboxes[1],bboxes[4],bboxes[5]]
    return [
         int(1000*(a[0] / width)),
         int(1000*(a[1] / height)),
         int(1000*(a[2] / width)),
         int(1000*(a[3] / height)),
     ]

def get_ocr_words_and_boxes(examples):
    
  # get a batch of document images
  images = [Image.open('/opt/ml/docvqa/data/train/' + image_file).convert("RGB") for image_file in examples['image']]
  # resize every image to 224x224 + apply tesseract to get words + normalized boxes
  encoded_inputs = feature_extractor(images)
  input_words = []
  bboxes = []
  for id in examples['image']:
    tmp = id[10:-4] + '.json'
    input_word = []
    bbox = []
    with open(f'/opt/ml/docvqa/data/train/ocr_results/{tmp}') as f:
      words = json.load(f)
    width = words['recognitionResults'][0]['width']
    height = words['recognitionResults'][0]['height']
    for i in range(len(words['recognitionResults'][0]['lines'])):
      l = len(words['recognitionResults'][0]['lines'][i]['text'].split())
      tmp_box = normalize_bbox(words['recognitionResults'][0]['lines'][i]['boundingBox'], width, height) 
      input_word.extend(words['recognitionResults'][0]['lines'][i]['text'].split())
      bbox.extend([tmp_box]*l)
    input_words.append(input_word)
    bboxes.append(bbox)

  examples['image'] = encoded_inputs.pixel_values
  examples['words'] = input_words
  examples['boxes'] = bboxes

  return examples

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
dataset_with_ocr = dataset.map(get_ocr_words_and_boxes, batched=True, batch_size=32)

100%|██████████| 1234/1234 [53:49<00:00,  2.62s/ba]


In [37]:
dataset_with_ocr['train'][0]['boxes']

[[339, 14, 585, 62],
 [637, 16, 663, 26],
 [637, 16, 663, 26],
 [410, 99, 586, 116],
 [410, 99, 586, 116],
 [410, 99, 586, 116],
 [149, 145, 202, 159],
 [149, 145, 202, 159],
 [233, 129, 329, 164],
 [235, 163, 380, 202],
 [235, 163, 380, 202],
 [146, 202, 388, 221],
 [146, 202, 388, 221],
 [146, 202, 388, 221],
 [397, 197, 574, 221],
 [397, 197, 574, 221],
 [151, 233, 313, 248],
 [151, 233, 313, 248],
 [151, 233, 313, 248],
 [148, 261, 241, 279],
 [253, 257, 392, 280],
 [253, 257, 392, 280],
 [151, 322, 241, 336],
 [151, 322, 241, 336],
 [611, 351, 703, 369],
 [800, 352, 846, 370],
 [149, 381, 283, 398],
 [149, 381, 283, 398],
 [601, 373, 666, 397],
 [147, 411, 269, 427],
 [147, 411, 269, 427],
 [149, 439, 278, 456],
 [149, 439, 278, 456],
 [132, 465, 335, 483],
 [132, 465, 335, 483],
 [139, 495, 322, 508],
 [139, 495, 322, 508],
 [143, 520, 305, 536],
 [143, 520, 305, 536],
 [142, 550, 298, 562],
 [142, 550, 298, 562],
 [607, 552, 665, 578],
 [231, 618, 756, 636],
 [231, 618, 756, 636

In [38]:
dataset_with_ocr = dataset_with_ocr['train']

In [39]:
def answer(words_list, answer_list):  
    matches = []
    start_indices = []
    end_indices = []
    for idx, i in enumerate(range(len(words_list))):
        if words_list[i] == answer_list[0] and words_list[i:i+len(answer_list)] == answer_list:
            matches.append(answer_list)
            start_indices.append(idx)
            end_indices.append(idx + len(answer_list) - 1)
    if matches:
      return matches[0], start_indices[0], end_indices[0]
    else:
      return None, 0, 0

In [40]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/layoutlmv2-base-uncased")

In [129]:
i = 4
encoding = tokenizer(dataset_with_ocr[i]['question'], dataset_with_ocr[i]['words'],boxes = dataset_with_ocr[i]['boxes'])

In [130]:
match, word_idx_start, word_idx_end = answer(dataset_with_ocr[i]['words'], dataset_with_ocr[i]['answers'][0])

In [132]:
dataset_with_ocr[i]['answers']

['Review of existing Brainstorming Ideas/483']

In [128]:
print("Match:", match)
print("Answer:", dataset_with_ocr[i]['answers'])
print("Word idx start:", word_idx_start)
print("Word idx end:", word_idx_end)

Match: None
Answer: ['Review of existing Brainstorming Ideas/483']
Word idx start: 0
Word idx end: 0
